# Testbench Experiments
### Testbench
- Given some pipeline, and amount of data to train on?
- Fit pipeline
- Then read some 'test' games from pgn and predict elos using pipeline
- Then calculate the r2 of the predictions
- Output a 'report' file containing relevant info: description of the pipeline (named_steps), number of games used to train on, number of (unseen) games tested on, r2 score.
### Pipeline
- The models must be part of a pipeline, containing a series of transforms to extract features / scale data followed by an estimator (i.e. regression model)
- The input to the pipelines here will be a list of chess.Games X and a corresponding list of Elo ratings y

In [2]:
import chess
import chess.pgn
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
import time
import json

In [1]:

#TRAIN_FILE = '../data/train_50k.pgn'
TRAIN_FILE = '../data/std_train_big.clean.pgn'
#TEST_FILE = '../data/test_10k.pgn'
TEST_FILE = '../data/std_test_small.clean.pgn'

def test(pipe, train_count, test_count, filename, description=None):
    train_pgn = open(TRAIN_FILE)
    X_train = []
    y_train = []
    for i in range(train_count):
        game = chess.pgn.read_game(train_pgn)
        X_train.append(game)
        y_train.append([int(game.headers['WhiteElo']),int(game.headers['BlackElo'])])

    fit_start = time.time()
    pipe.fit(X_train, y_train)
    fit_end = time.time()
    fit_time = (fit_end - fit_start)
    
    test_pgn = open(TEST_FILE)
    X_test = []
    y_test = []
    for i in range(test_count):
        game = chess.pgn.read_game(train_pgn)
        X_test.append(game)
        y_test.append([int(game.headers['WhiteElo']),int(game.headers['BlackElo'])])
    
    pred_start = time.time()
    y_pred = pipe.predict(X_test)
    pred_end = time.time()
    pred_time = (pred_end - pred_start)

    R2 = r2_score(y_test, y_pred)
    MSE = mean_squared_error(y_test, y_pred)
    
    results = {
        'Description': description,
        'Pipeline': str(pipe.named_steps),
        '# Games for training': train_count,
        '# Games for testing': test_count,
        'Fit time': fit_time,
        'Predict time': pred_time,
        'R2 score': R2,
        'MSE': MSE
    }
    print(results)
    with open(f'../reports/{filename}.json', 'w') as file:
        json.dump(results, file)

### Example
- Making & testing a pipeline with game2vec for feature extraction and knn regressor as estimator

In [3]:
# From chess_utils.py

def board_to_vec(board):
    '''
        Given a chess.Board return a vector of length 64
        representing the piece / lack of piece at a given square.
    '''
    vec = np.zeros((64), dtype=int)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            if piece.color == chess.WHITE:
                vec[square] = piece.piece_type
            else:
                vec[square] = -1 * piece.piece_type
    return vec

def game_to_vec(game, moves_limit):
    '''
    Given a chess.Game, return a concatenation of board states
    represented as vectors, as generated by board_to_vec()
    '''
    board = game.board()
    game_as_vec = np.zeros((64 * moves_limit))
    i = 0
    for move in game.mainline_moves():
        if i >= moves_limit:
            break
        board.push(move)
        game_as_vec[(64*i):(64*(i+1))] = board_to_vec(board)
        i += 1
    return game_as_vec

# From knn_model.py

def games_to_opening_vecs(games):
    return np.array(list(map(lambda game: game_to_vec(game, 35), games)))

knn_model = KNeighborsRegressor(n_neighbors = 12, weights = 'uniform', metric='hamming')
knn_pipe = Pipeline([
    ('Game to vec', FunctionTransformer(games_to_opening_vecs)),
    ('kNN', knn_model)
     ])

testbench(knn_pipe, 1000, 100, 'knn_report', 'KNN model with hamming window metric gets good performance with m=35, k=12')